(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction) click here to [join](https://kaggle.com/join/datascience16helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

In [1]:
# Libraries
import numpy as np
import pandas as pd

import gzip
from collections import defaultdict

import time

In [2]:
def readJson(f):
  for l in open(f):
    yield eval(l)

now = time.time()
usersitem = defaultdict(dict)

rate = {}

for l in readJson("train.json"):
    
    usersitem[l['reviewerID']][l['asin']] = l['overall']

print "Done", len(usersitem), time.time()-now


Done 509678 46.6509859562


In [19]:
now = time.time()
itemsattr = defaultdict(list)

attrset = set()

for l in readJson("meta.json"):
    categ = l['categories'].translate(None,'\'').split("[[")[1].split("]]")[0].split(",")[:1]
    itemsattr[l['asin']] = categ
    attrset |= set(categ)
    
print "Done", time.time()-now, len(itemsattr), len(attrset)

Done 9.05023980141 498196 9


In [20]:
print itemsattr['03975a07fc9d5777a251e73cd7421aff026c7c5d3d58b7d66fae6d0b9d48ff7a']
print usersitem['bc19970fff3383b2fe947cf9a3a5d7b13b6e57ef2cd53abc52bb2dfedf5fb1cd']['19e5cc4a706554d37670eabca2c19f1fc4f259361d78f0b58dafb91f3a863fc1']

['Electronics']
2.0


In [21]:
import sys

userfeature = defaultdict(dict)
itemfeature = defaultdict(dict)

avgratedict = defaultdict(list)
useroffdict = defaultdict()

start = time.time()

def populateAvgDict():
    for user in usersitem.keys():
        for item in usersitem[user].keys():
            #print user, usersitem[user], item, itemsattr[item]
            avgratedict[item].append(usersitem[user][item])
            #for feature in list(attrset):
            for feature in itemsattr[item]:
                #if feature in itemsattr[item]:
                userfeature[user][feature] = usersitem[user][item]
                itemfeature[item][feature] = usersitem[user][item]
                """
                else:
                    userfeature[user][feature] = 0.0
                    itemfeature[item][feature] = 0.0
                """

                """
                if len(userfeature) == 5:
                    print userfeature, itemfeature
                    sys.exit(0)
                """    
    return avgratedict
        
avgratedict = populateAvgDict()  
print len(userfeature), len(itemfeature), time.time()-start

509678 171185 3.34739995003


In [22]:
print userfeature['bc19970fff3383b2fe947cf9a3a5d7b13b6e57ef2cd53abc52bb2dfedf5fb1cd']
print itemfeature['03975a07fc9d5777a251e73cd7421aff026c7c5d3d58b7d66fae6d0b9d48ff7a']


{'Electronics': 2.0}
{'Electronics': 3.0}


In [23]:
globalsum = 0
globalsize = 0

for k,v in avgratedict.iteritems():
    globalsum += sum(v)
    globalsize += len(v)
    
globalavg = globalsum/globalsize

print globalsum, globalsize, globalavg


3837538.0 1000000 3.837538


In [24]:
useroffset = defaultdict(list)
for user,items in usersitem.iteritems():
    for item, rating in items.iteritems():        
        useroffset[user].append(usersitem[user][item] -  sum(avgratedict[item])/len(avgratedict[item]))
print "Computed users' offset", len(useroffset)
     

Computed users' offset 509678


In [25]:
from scipy.stats import logistic

#head start before SVD


def SVDBase(userid, itemid):
    ret = 0.0
    try:
        rateavg = avgratedict[itemid]
        offset = useroffset[userid] 
        #if not len(rateavg) or not len(offset):
            #print userid, itemid
        if len(rateavg):
            itemavg = (globalavg*25 + sum(rateavg))/(25 + len(rateavg))
        else:
            itemavg = globalavg
        if len(offset):
            offsetavg = sum(offset)/len(offset)
        else:
            offsetavg = 0.0

        ret =  min(5.0, max(0.5,itemavg+offsetavg))

    except KeyError:
        ret = min(5.0, max(0.5, globalavg))
        
    return ret


ratefile = open("preSVDpred_Rating.txt","w")
ratefile.write("reviewerID-asin,prediction\n")
start = time.time()

reqUser = defaultdict(dict)
reqItem = defaultdict(dict)
reqAttr = set()

with open("pairs_Rating.txt") as prate:
    for pair in prate:
        #print pair
        if pair.startswith("reviewer"):
            continue
        
        userid, itemid = pair.split("-")
        itemid = itemid[:-1]
        basepredict = SVDBase(userid,itemid)                
        
        reqUser[userid][itemid] = basepredict
        reqItem[itemid][userid] = basepredict
        reqAttr |= set(itemsattr[itemid])
        
        ratefile.write("%s-%s,%f\n" % (userid, itemid, basepredict))
        #sys.exit(0)
ratefile.close()

        
print "Done", time.time()-start, len(reqUser), len(reqItem), len(reqAttr)

Done 1.27287697792 88257 47289 4


## DATA REDUCTION

In [26]:
attrlist = sorted(reqAttr)
userlist = list(reqUser.keys())
itemlist = list(reqItem.keys())

r = [0.0 for _ in xrange(len(reqUser))]
userfeat = [r for _ in xrange(len(attrlist)) ]

r = [0.0 for _ in xrange(len(reqItem))]
itemfeat = [r for _ in xrange(len(attrlist)) ]
print len(userfeat), len(userfeat[0]), len(itemfeat), len(itemfeat[0])

start = time.time()
#print list(reqUser.keys()).index("f0ce42c52f549e542b28cb6351b93814be2c571809bca8eab2e191e601ada746")

for f in xrange(len(attrlist)):
    for u in xrange(len(userfeat[f])):        
        try:
            userfeat[f][u] = userfeature[userlist[u]][attrlist[f]]
        except:
            userfeat[f][u] = 0.0
    
    for i in xrange(len(itemfeat[f])):
        try:
            itemfeat[f][i] = itemfeature[itemlist[i]][attrlist[f]]
        except:
            itemfeat[f][i] = 0.0
    #break
print "Done", time.time()-start            
U = np.array(userfeat)
V = np.array(itemfeat)

print U.shape, V.shape


4 88257 4 47289
Done 0.726974964142
(4, 88257) (4, 47289)


In [27]:
print U[11]
print V[20]

IndexError: index 11 is out of bounds for axis 0 with size 4

 BEGIN TRAINING

In [28]:
start = time.time()

def predictRating(user,item):
    ret = 0.0    
    for f in xrange(len(attrlist)):
        ret += max(0.0, min(5.0, U[f][user]*V[f][item]))
    
    return ret
        
def train(f, user, item, rating, lrate=0.002, K=0.01):
    err = lrate*(rating - predictRating(user,item))
    uv = U[f][user]
    
    U[f][user] += lrate*(err*V[f][item] - K*U[f][user])
    V[f][item] += lrate*(err*U[f][user] - K*V[f][item])   

    
for f in xrange(len(attrlist)):
    for u in xrange(len(userlist)):   
        if userlist[u] not in usersitem:
            continue
            
        for i in xrange(len(itemlist)):
            try:
                train(f, u, i, usersitem[userlist[u]][itemlist[i]])
            except KeyError:
                continue
    break
            
print "Time", time.time()-start

KeyboardInterrupt: 

-----------------

### BEGIN SVD

In [2]:
from recsys.algorithm.factorize import SVD
svd = SVD()


ImportError: No module named recsys.algorithm.factorize

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()